In [15]:
# import sys
# from pathlib import Path

# # Notebook cell
# %load_ext autoreload
# %autoreload 2

# # Get root directory (assuming notebook is in root/notebooks/)
# NOTEBOOK_DIR = Path.cwd()
# ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# # Add src directory to Python path
# sys.path.append(str(ROOT_DIR / 'src'))

# # Verify path
# print(f"Python will look in these locations:\n{sys.path}")


# # --- Execute the processor ---
# import utils


# SOURCE_PATH, DEST_PATH = utils.main_processor(
#     data_dir='..\data',  # search project ..\data    
#     downloads_dir=None,  # None searchs Downloads dir, '' omits search
#     downloads_limit=50,  # search the first 10 files
#     clean_name_override=None,  # override filename
#     start_file_pattern='df_OHLCV_', # search for files starting with 'df_'
# )



In [16]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")
print(f"DOWNLOAD_DIR: {DOWNLOAD_DIR}")
print(f"DEST_DIR: {DEST_DIR}\n")

# Build paths
source_path = Path(DOWNLOAD_DIR) / f'df_OHLCV_{date_str}.parquet'
dest_path = Path(DEST_DIR) / f'df_OHLCV_{date_str}_clean.parquet'

print(f"source_path: {source_path}")
print(f"dest_path: {dest_path}")

date_str: 2025-04-01
DOWNLOAD_DIR: C:\Users\ping\Downloads
DEST_DIR: ..\data

source_path: C:\Users\ping\Downloads\df_OHLCV_2025-04-01.parquet
dest_path: ..\data\df_OHLCV_2025-04-01_clean.parquet


In [17]:
# --- Data Loading & Initial Inspection ---
import pandas as pd

# # Load raw data from pickle file
# df = pd.read_pickle(source_path)

df = pd.read_parquet(source_path, engine='pyarrow')

# Display initial data structure
print("[Raw Data Overview]")
display(df.head())
df.info()

[Raw Data Overview]


Open   High    Low  Close  Adj Close   Volume  Adj Open  \
Symbol Date                                                                   
TBIL   2025-04-01  49.84  49.85  49.83  49.84      49.84  3609452     49.84   
       2025-03-31  50.00  50.01  50.00  50.01      50.01  3381800     50.00   
       2025-03-28  50.00  50.01  49.99  49.99      49.99  2638500     50.00   
       2025-03-27  49.98  49.99  49.98  49.99      49.99  2561300     49.98   
       2025-03-26  49.97  49.98  49.97  49.98      49.98  1540200     49.97   

                   Adj High  Adj Low  
Symbol Date                           
TBIL   2025-04-01     49.85    49.83  
       2025-03-31     50.01    50.00  
       2025-03-28     50.01    49.99  
       2025-03-27     49.99    49.98  
       2025-03-26     49.98    49.97

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 389418 entries, ('TBIL', Timestamp('2025-04-01 00:00:00')) to ('IEI', Timestamp('2024-04-02 00:00:00'))
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Open       389418 non-null  float64
 1   High       389418 non-null  float64
 2   Low        389418 non-null  float64
 3   Close      389418 non-null  float64
 4   Adj Close  389418 non-null  float64
 5   Volume     389397 non-null  Int64  
 6   Adj Open   389418 non-null  float64
 7   Adj High   389418 non-null  float64
 8   Adj Low    389418 non-null  float64
dtypes: Int64(1), float64(8)
memory usage: 28.6+ MB


In [18]:
import sys
from pathlib import Path

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))


# --- Data Filtering & Cleaning ---
import utils  # Custom utility functions

# 1. Align dates across all symbols using SPY as reference
df = utils.filter_df_dates_to_reference_symbol(df=df, reference_symbol='SPY')

# 2. Remove symbols with missing data points
df_clean, missing_symbols = utils.filter_symbols_with_missing_values(df)

# Display cleaning results
print("\n[Cleaning Report]")
print(f"Removed {len(missing_symbols)} symbols with missing data: {missing_symbols}")
print("\n[Cleaned Data Structure]")
df_clean.info()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Original number of symbols: 1559
Number of symbols after filtering: 1540
Number of symbols filtered out: 19

First 10 symbols that were filtered out:
['BTC', 'VIK', 'WAY', 'TTAN', 'SAIL', 'VG', 'ZK', 'STRK', 'SFD', 'LTM']

Example of dates for first filtered out symbol:

Dates for BTC:
DatetimeIndex(['2025-04-01', '2025-03-31', '2025-03-28', '2025-03-27',
               '2025-03-26', '2025-03-25', '2025-03-24', '2025-03-21',
               '2025-03-20', '2025-03-19',
               ...
               '2024-08-13', '2024-08-12', '2024-08-09', '2024-08-08',
               '2024-08-07', '2024-08-06', '2024-08-05', '2024-08-02',
               '2024-08-01', '2024-07-31'],
              dtype='datetime64[ns]', name='Date', length=168, freq=None)

Filtered DataFrame info:
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 386540 entries, ('TBIL', Timestamp('2025-04-01 00:00:00')) to ('IEI', Timestamp('2024

In [19]:
# --- Save Cleaned Data ---
# # Save processed data to pickle file
# df_clean.to_pickle(dest_path)

# Using PyArrow (default, recommended for most cases)
df_clean.to_parquet(dest_path, engine='pyarrow')
print(f"\n[Save Successful] Cleaned data saved to:\n{dest_path}")




[Save Successful] Cleaned data saved to:
..\data\df_OHLCV_2025-04-01_clean.parquet
